In [1]:
import pandas as pd
import requests
import folium
import geopandas as gpd
import shapely
from shapely.geometry import Polygon

This notebook loads the FSIS data, filters it for large poultry processors, calls the Mapbox API for driving distance isochrones, and pickles it so it can be directly loaded into other notebooks.

In [2]:
PICKLE_PATH = "../data/pickled-chickens.pickle"
FSIS_DATA = "../data/fsis-processors-with-location.csv"

In [3]:
df = pd.read_csv(FSIS_DATA)
df.head()

,Unnamed: 0,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude
0,0,M267,4802,JBS,JBS Tolleson Inc.,AZ,Large,Beef,5.0,4.0,"651 S. 91st Ave, Tolleson, AZ 85353",33.441660,-112.252559
1,1,M354,5117,Cargill,Cargill Meat Solutions,CA,Large,Beef,5.0,4.0,"3115 S. Fig Ave., Fresno, CA 93706",36.689651,-119.801950
2,2,M6063A,5289,Central Valley Meat Company,"Central Valley Meat Co., Inc.",CA,Large,Beef,5.0,4.0,"10431 8 3/4 Ave., Hanford, CA 93230",36.321273,-119.612222
3,3,M783 + P783 + V783,5144,Central Valley Meat Company,Harris Ranch Beef Company,CA,Large,Beef,5.0,4.0,"16277 S McCall Ave., Selma, CA 93662",36.499212,-119.614553
4,4,M21488 + V21488,8349,Independent,OWB Packers LLC,CA,Large,Beef,4.0,4.0,"57 East Shank Road, Brawley, CA 92227",33.000969,-115.521786


In [4]:
len(df)

1149

In [6]:
df_chickens = df[df['Animals Processed'].str.contains('Chicken')]
len(df_chickens)

258

In [7]:
df_large_chickens = df_chickens.loc[df_chickens.Size == "Large"]
df_large_chickens.head()

,Unnamed: 0,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude
71,71,P1317 + V1317,4495,Cargill,Wayne Farms LLC,AL,Large,Chicken,5.0,5.0,"700 McDonald Avenue, Albertville, AL 35950",34.260726,-86.203222
72,72,P7485 + V7485,4518,Cargill,Wayne Farms LLC,AL,Large,Chicken,5.0,5.0,"1020 County Road 114, Jack, AL 36346",31.500628,-85.903438
73,73,P912,4493,Cargill,"Wayne Farms, LLC",AL,Large,Chicken,5.0,5.0,"444 Baskin Street South, Union Springs, AL 36089",32.139166,-85.721704
74,74,P7342 + V7342,4516,Cargill,Wayne Farms LLC,AL,Large,Chicken,5.0,5.0,"808 Ross Clark Circle NE, Dothan, AL 36303",31.225754,-85.362068
75,75,P1235,4509,Cargill,Wayne Farms LLC,AL,Large,Chicken,5.0,5.0,"254 Ipsco Road, Decatur, AL 35601",34.607601,-87.042595


In [8]:
len(df_large_chickens)

138

### Save Isochrones to Dataframe

In [9]:
# mapbox constants
MAPBOX_TOKEN = "pk.eyJ1IjoidG9kZG5pZWYiLCJhIjoiY2xncGpzbmhhMTBwdzNnbXJjNWlzaTY2aCJ9.UhUELBA2iNIDsTN9YESsIw"
ENDPOINT = "https://api.mapbox.com/isochrone/v1/mapbox/driving/"
DRIVING_DISTANCE = str(int(60 * 1609.34)) # 60 miles in meters: 90 percent of all birds were produced on farms within 60 miles of the plant, according to 2011 ARMS data

In [ ]:
isochrones = []
for index, row in df_large_chickens.iterrows():
    lat = str(row['latitude'])
    lng = str(row['longitude'])

    # add driving radius isochrone to map
    url = ENDPOINT + lng + "," + lat + "?" + "contours_meters=" + DRIVING_DISTANCE + "&access_token=" + MAPBOX_TOKEN
    response = requests.get(url)
    isochrone = Polygon(response.json()['features'][0]['geometry']['coordinates'])
    isochrones.append(isochrone)

In [ ]:
df_large_chickens = df_large_chickens.copy()
df_large_chickens["Isochrone"] = isochrones

In [ ]:
df_large_chickens.head()

In [ ]:
df_large_chickens.to_pickle(PICKLE_PATH)